# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

# Creating path and importing Dataframe

In [2]:
path = r'C:\Users\stefa\15-02-2022 Instacart Basket Analysis'

In [3]:
df_customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

# 4. Wrangle the data so that it follows consistent logic; for example, rename columns with illogical names and drop columns that don’t add anything to your analysis.

In [4]:
df_customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [5]:
df_customers.shape

(206209, 10)

In [6]:
df_customers.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [7]:
# With the above Basics Checks we can identify Typos, wrong column names and set up the following Code for renaming

In [8]:
df_customers = df_customers.rename(columns = {'First Name':'first_name', 'Surnam':'surname', 'Gender':'gender', 'STATE':'state', 'Age':'age'})

In [9]:
# I changed the columns to lower case letters and edited typos

# 5. Complete the fundamental data quality and consistency checks you’ve learned throughout this Achievement; for example, check for and address missing values and duplicates, and convert any mixed-type data.

In [10]:
# In order to check for data that can be dropped , i used the Code to find missing columns

In [11]:
df_customers.isnull().sum()

user_id             0
first_name      11259
surname             0
gender              0
state               0
age                 0
date_joined         0
n_dependants        0
fam_status          0
income              0
dtype: int64

In [12]:
df_customers = df_customers.drop('first_name', 1)

C:\Users\stefa\AppData\Local\Temp/ipykernel_15336/2502238926.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_customers = df_customers.drop('first_name', 1)


In [13]:
# We can see that the colum with first names has 11259 missing Values. Hence there could be problems if there are different customers with the same last name, i decided to drop this column as every customer still has a unique user_id

In [14]:
# I checked for duplicates in our Dataframe which might alter the analysis

In [15]:
df_duplicate = df_customers[df_customers.duplicated()]

In [16]:
df_duplicate

,user_id,surname,gender,state,age,date_joined,n_dependants,fam_status,income


In [17]:
# I did not find any duplicates in my Dataset

In [18]:
# And last but not least i checked for mixed Datatypes as described in previous Achievements

In [19]:
for col in df_customers.columns.tolist():
  weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customers[weird]) > 0:
    print (col)

In [20]:
# There are no mixed Datatypes identified.

# 6.Combine your customer data with the rest of your prepared Instacart data. Tip: Make sure the key columns are of the same data type! Hint: Make sure the key columns are the same data type!

In [21]:
# Importing Dataframe from Exercise 4.8

ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_mergedV4.pkl'))

In [22]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_time,days_since_prior_order,First_order,product_id,add_to_cart_order,reordered,...,price_range_loc,busiest_day,busiest_days,Busiest_period_of_day,max_order,loyalty_flag,spending,spending_flag,frequency,frequency_flag
0,2539329,1,1,2,8,NaN,True,196.0,1.0,0.0,...,Mid-range product,Regularly busy,Regularly busy,Average_orders,11,Regular customer,6.367797,Low Spender,20.0,Regular customer
1,2539329,1,1,2,8,NaN,True,14084.0,2.0,0.0,...,Mid-range product,Regularly busy,Regularly busy,Average_orders,11,Regular customer,6.367797,Low Spender,20.0,Regular customer
2,2539329,1,1,2,8,NaN,True,12427.0,3.0,0.0,...,Low-range product,Regularly busy,Regularly busy,Average_orders,11,Regular customer,6.367797,Low Spender,20.0,Regular customer
3,2539329,1,1,2,8,NaN,True,26088.0,4.0,0.0,...,Low-range product,Regularly busy,Regularly busy,Average_orders,11,Regular customer,6.367797,Low Spender,20.0,Regular customer
4,2539329,1,1,2,8,NaN,True,26405.0,5.0,0.0,...,Low-range product,Regularly busy,Regularly busy,Average_orders,11,Regular customer,6.367797,Low Spender,20.0,Regular customer


In [23]:
# Since the user ID is present in both Sets, we can use it to combine our two Datasets ords_prods_merge and our newly  df_customers

In [24]:
# In order to verify the sucessfull merge we have to drop the _merge Colum from ords_prods_merge first.

In [25]:
pd.options.display.max_rows = None

In [26]:
pd.options.display.max_columns = None

In [27]:
ords_prods_merge = ords_prods_merge.drop('_merge', 1)

C:\Users\stefa\AppData\Local\Temp/ipykernel_15336/933980236.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  ords_prods_merge = ords_prods_merge.drop('_merge', 1)


In [28]:
ords_prods_customers = ords_prods_merge.merge(df_customers, on = 'user_id', indicator = True)

# Ensure your notebook contains logical titles, section headings, and descriptive code comments.

In [29]:
# done during the task

# Export this new dataframe as a pickle file so you can continue to use it in the second part of this task.

In [30]:
ords_prods_customers.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_customers.pkl'))

In [32]:
ords_prods_customers.shape

(32641268, 33)